# Decorators
---

### Introduction

The decorator's operation is based on wrapping the original function with a new "decorating" function (or class), hence the name "decoration". Of course, the decorating function does more, __because it can take the parameters of the decorated function and perform additional actions__ and that make it a real decorating function.

__NOTE__: Note that decorators, in Python, are still closures.

A decorator, apart from encapsulating a function, can also receive the arguments of that function and receive other arguments not related to it. The following are some of the possibilities with decorators.

Here are some case uses of decorators:
- Working with arguments:
    - Validation
    - Modification before passing to the decorated function
    - Return the result arguments parsed
- Debugging
    - Measurement of execution time.
    - Message logging.
- Thread synchronization.
- DRY (Don't Repeat Yourself) principle.
- Caching.

---

### Decorators: Only wrapping a function

This is the simplest example where the decorator encapsulates a function and does not need to send arguments to it (since it does not use them).

Here goes a code example:

In [10]:
def simple_decorator(func):  # The argument of a decorator it's always the wrapped function
    print(f"The simple_decorator is wrapping {func.__name__}")
    return func  # The last argument of a decorator is called
    # ^ It passes all the arguments send by the user even without concreting

@simple_decorator  # This can be interpreted as simple_decorator(simple_function())
def simple_function(arg1, arg2):  # This function now is received as an argument by the simple_decorator
    print(f"The parameters received are '{arg1}' and '{arg2}'")

simple_function("Hi", "Bye")


The simple_decorator is wrapping simple_function
The parameters received are 'Hi' and 'Bye'


---

### Decorators: Working with the function arguments

One of the examples given above is that the decorator can work with arguments of the function it encapsulates, however, in the above demonstration we do not work with these arguments, so to do so we must use closures to access these arguments.

An example in code would be the following:

In [11]:
def simple_decorator(func):
    def wrapper(*args, **kwargs):  # Now i have access to the arguments sent by the user
        print(f"The simple_decorator is wrapping {func.__name__}")
        print(f"The decorator has access to {args[0]} and {args[1]}")  # We are using the *args parameters because in this example we haven't used keywords
        print(f"Argument that the decorated function doesn\'t use '{kwargs['decorator_argument']}'")
        return func(*args, **kwargs)  # In this section the encapsulated function must be called manually by sending the received arguments
    return wrapper  # We return the wrapper itself so that it receives the arguments sent to the function

@simple_decorator
def simple_function(arg1="", arg2="", **kwargs):
    print(f"The parameters received are '{arg1}' and '{arg2}'")

simple_function("Hi", "Bye", decorator_argument="Yoh!")

The simple_decorator is wrapping simple_function
The decorator has access to Hi and Bye
Argument that the decorated function doesn't use 'Yoh!'
The parameters received are 'Hi' and 'Bye'



---

### Decorator: Receiving it's own arguments

The decorator can also receive arguments, which are not related to the function it encapsulates, but are used by the decorator itself. This is done by creating a function that returns the decorator, which in turn encapsulates the function.

Let's create an example that validates the datatype of the arguments received by the wrapped function:

In [20]:
def validate_datatype(validate_datatype):  # This example just adds an initial function that receive the specified decorator argument
    def simple_decorator(func):
        def wrapper(*args, **kwargs):
            print(f"The simple_decorator is wrapping {func.__name__}")
            print(f"The decorator has access to {args[0]} and {args[1]}\n")
            print(f"The decorator checks that the arguments received have the datatype '{str(validate_datatype)}':")
            for arg in args:
                print(f"The argument '{arg}' ", end="")
                if not isinstance(arg, validate_datatype):
                    print(f"doesn't have the expected datatype (has {str(type(arg))})")
                else:
                    print("have the expected datatype")
            print(f"\nArgument that the decorated function doesn\'t use '{kwargs['decorator_argument']}'")
            return func(*args, **kwargs)
        return wrapper
    return simple_decorator

@validate_datatype(int)  # The argument is passed directly from here
def simple_function(arg1="", arg2="", **kwargs):
    print(f"The parameters received are '{arg1}' and '{arg2}'")

simple_function("Hi", 2, decorator_argument="Yoh!")

The simple_decorator is wrapping simple_function
The decorator has access to Hi and 2

The decorator checks that the arguments received have the datatype '<class 'int'>':
The argument 'Hi' doesn't have the expected datatype (has <class 'str'>)
The argument '2' have the expected datatype

Argument that the decorated function doesn't use 'Yoh!'
The parameters received are 'Hi' and '2'


---

### Stacking decorators

You can accumulate as many decorators as you wish, however, keep in mind that the order in which they are included indicates the order in which they will be executed.

The syntax for stacking decorators is as follows:

In [17]:
def outer_decorator(func):
    def wrapper():
        print("Outer time!")
        func()
    return wrapper


def inner_decorator(func):
    def wrapper():
        print("Inner time!")
        func()
    return wrapper

@outer_decorator
@inner_decorator
def func():
    print("Function time!")


func()
    

Outer time!
Inner time!
Function time!


Adding the decorators is the same as calling the function like this:

```outer_decorator(inner_decorator(func()))```

---